[Reference](https://blog.devgenius.io/data-engineering-concepts-4-distributed-systems-and-the-master-worker-architecture-a61dc238b4ba)

# Deploying MongoDB Cluster
```
version: '3.8'

networks:
  default:
    name: mongoCluster

services:
  mongodb_master:
    image: mongo:latest
    container_name: mongodb_master
    restart: always
    ports:
      - 27017:27017
    volumes:
      - ./master_data:/data/db
    networks:
      - default
    command: mongod --replSet my_cluster --bind_ip localhost,mongodb_master

  mongodb_worker1:
    image: mongo:latest
    container_name: mongodb_worker1
    restart: always
    ports:
      - 27018:27017
    volumes:
      - ./worker1_data:/data/db
    networks:
      - default
    command: mongod --replSet my_cluster --bind_ip localhost,mongodb_worker1

  mongodb_worker2:
    image: mongo:latest
    container_name: mongodb_worker2
    restart: always
    ports:
      - 27019:27017
    volumes:
      - ./worker2_data:/data/db
    networks:
      - default
    command: mongod --replSet my_cluster --bind_ip localhost,mongodb_worker2
```

```
docker-compose up -d
```

```
docker exec -it mongodb_master mongosh --eval "rs.initiate({
 _id: \"my_cluster\",
 members: [
   {_id: 0, host: \"mongodb_master\"},
   {_id: 1, host: \"mongodb_worker1\"},
   {_id: 2, host: \"mongodb_worker2\"}
 ]
})"
```

```
#!/bin/bash

docker-compose up -d

bash execute.sh
```

# Using the Cluster
```python
from pymongo import MongoClient

# Connecting to the master as a read and write primary node
master_client = MongoClient("mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+1.10.5&readPreference=primary")

# Connect to worker1 as a read-only secondary node
worker1_client = MongoClient("mongodb://127.0.0.1:27018/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+1.10.5&readPreference=secondary")

# Connect to worker2 as a read-only secondary node
worker2_client = MongoClient("mongodb://127.0.0.1:27019/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+1.10.5&readPreference=secondary")
```

```python
print(master_client.list_database_names())
# Output: ['admin', 'config', 'local']
print(master_client.read_preference)
print(worker1_client.read_preference)
print(worker2_client.read_preference)
# Primary()
# Secondary(tag_sets=None, max_staleness=-1, hedge=None)
# Secondary(tag_sets=None, max_staleness=-1, hedge=None)

# Creating a database called family
family = master_client.get_database("family")

# Creating a collection called members
members = family.create_collection("members")

# Print all databases
master_client.list_database_names()

# Output: ['admin', 'config', 'family', 'local']

# Print all databases on worker1 and worker2
worker1_client.list_database_names()
worker2_client.list_database_names()

# Output: ['admin', 'config', 'family', 'local']
# Output: ['admin', 'config', 'family', 'local']

# Grab the family database
family = master_client.get_database("family")

# Grab the members collection
members = family.get_collection("members")

# Insert 5 members
members.insert_many([
    {"name": "Bar", "age": 31},
    {"name": "Yuval", "age": 30},
    {"name": "Dekel", "age": 32},
    {"name": "Amos", "age": 60},
    {"name": "Ronit", "age": 60}
])

# Print all members
for member in members.find():
    print(member)


# {'_id': ObjectId('6500a9d252d01d79c5f44fbf'), 'name': 'Bar', 'age': 31}
# {'_id': ObjectId('6500a9d252d01d79c5f44fc0'), 'name': 'Yuval', 'age': 30}
# {'_id': ObjectId('6500a9d252d01d79c5f44fc1'), 'name': 'Dekel', 'age': 32}
# {'_id': ObjectId('6500a9d252d01d79c5f44fc2'), 'name': 'Amos', 'age': 60}
# {'_id': ObjectId('6500a9d252d01d79c5f44fc3'), 'name': 'Ronit', 'age': 60}
```